In [ ]:
import pandas as pd
import uproot
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource

%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

import numpy as np

import functions_giammi as fugi
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()


points = pd.read_excel('../PYTHON_graphs/DATA/Energycalibration_2020-06-14.xlsx',
                  sheet_name = 'with_stretch',
                  header = 22,
                  nrows = 9,
                  usecols = "C, D",
                  convert_float = True)
                #   dtype={"B" : float, 'tof_rand': float, 'z_rand': float, 'mom_mag_rand': float,'cos(theta)_tof_rand': float})
points.dropna(inplace=True);

dir = r"../PYTHON_graphs/OUTPUTS/Ar_calibration/"
cmap_viridis = mpl.cm.get_cmap('viridis')



In [ ]:
points

In [ ]:
#polynomial fit with degree = 2
p, residuals, rank, singular_values, rcond =np.polyfit(points["ph energy"], points["radius square"], 2, full=True)
model = np.poly1d(p)

ybar = np.sum(points["radius square"])/len(points["radius square"]) 
sstot = np.sum((points["radius square"] - ybar)**2)
r2 = 1-np.divide(residuals,sstot)

#add fitted polynomial line to scatterplot
polyline = np.linspace(249, 261, 50)
fit_info = (f"y = {model[0]:+.2f} x\u00b2 {model[1]:+.2f}x {model[2]:+.2f} \nR\u00b2 = {r2[0]:.6f}")

fig,ax= plt.subplots(1,figsize=(10, 10))
# plt.suptitle("Fitting")
ax.scatter(points["ph energy"], points["radius square"], color=cmap_viridis(0), s=80)
ax.plot(polyline, model(polyline), "--", color=cmap_viridis(0), alpha=0.5)
ax.set_xlabel('photoelectron energy [eV]')
ax.set_ylabel('squared radius [mm\u00b2]')
ax.legend(title=fit_info, loc="best")

ax.grid(linestyle = '-', linewidth = 2 , alpha = 0.4)

fig.savefig(dir+"regression_radius.svg", dpi=300, transparent=False, bbox_inches='tight')
fig.savefig(dir+"regression_radius.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
Ar = uproot.open(r"../PYTHON_graphs/DATA/Experiments/Ar_FULL_analysis.root")
# Ar["all_toghether/"].keys()
# Ar["all_toghether/"].classnames()
# Ar["all_toghether/fish filet y"].all_members
# Ar["all_toghether/fish filet y"].get_values()
# type(Ar["all_toghether/fish filet y"])

In [ ]:
xmm_pos,ymm_pos,counts_252=fugi.import_TH2Dgeneric(Ar,"Ar_252eV_CR/electrons/hit_0/raw/position")
xmm_pos,ymm_pos,counts_254=fugi.import_TH2Dgeneric(Ar,"Ar_254eV_CR/electrons/hit_0/raw/position")
xmm_pos,ymm_pos,counts_256=fugi.import_TH2Dgeneric(Ar,"Ar_256eV_CR/electrons/hit_0/raw/position")
xmm_pos,ymm_pos,counts_260=fugi.import_TH2Dgeneric(Ar,"Ar_260eV_CR/electrons/hit_0/raw/position")
tof,xmm,fish_fillet_x=fugi.import_TH2Dgeneric(Ar,"all_toghether/fish filet x")
tof,ymm,fish_fillet_y=fugi.import_TH2Dgeneric(Ar,"all_toghether/fish filet y")
ctheta_en,en,cthetaEE=fugi.import_TH2Dgeneric(Ar,"all_toghether/check_cthetaEE")

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))

counts_tot=counts_252+counts_254+counts_256+counts_260

#masking
counts_tot_masked = np.ma.masked_array(counts_tot, counts_tot < 5)

# cs=ax.contourf(tof,ymm,fish_fillet_y_masked.T, levels=20)
cs=ax.pcolormesh(ymm_pos,xmm_pos,counts_tot_masked.T, shading='gouraud')

# ax.set_title("fish fillet y-axis")
ax.set_xlabel('x-axis [mm]')
ax.set_xlim([-62., 62])
ax.set_ylabel('y-axis [mm]')
ax.set_ylim([-62., 62.])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([fish_fillet_y_masked.min(),fish_fillet_y_masked.max()])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"position_allen.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"fish_fillet_y.svg", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))

counts_tot=counts_252

#masking
counts_tot_masked = np.ma.masked_array(counts_tot, counts_tot < 2)

# cs=ax.contourf(tof,ymm,fish_fillet_y_masked.T, levels=20)
cs=ax.pcolormesh(ymm_pos,xmm_pos,counts_tot_masked.T, shading='gouraud')

# ax.set_title("fish fillet y-axis")
ax.set_xlabel('x-axis [mm]')
ax.set_xlim([-62., 62])
ax.set_ylabel('y-axis [mm]')
ax.set_ylim([-62., 62.])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([fish_fillet_y_masked.min(),fish_fillet_y_masked.max()])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"position_en252ev.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"fish_fillet_y.svg", dpi=300, transparent=False, bbox_inches='tight')

plt.close()

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))

counts_tot=counts_254

#masking
counts_tot_masked = np.ma.masked_array(counts_tot, counts_tot < 2)

# cs=ax.contourf(tof,ymm,fish_fillet_y_masked.T, levels=20)
cs=ax.pcolormesh(ymm_pos,xmm_pos,counts_tot_masked.T, shading='gouraud')

# ax.set_title("fish fillet y-axis")
ax.set_xlabel('x-axis [mm]')
ax.set_xlim([-62., 62])
ax.set_ylabel('y-axis [mm]')
ax.set_ylim([-62., 62.])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([fish_fillet_y_masked.min(),fish_fillet_y_masked.max()])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"position_en254ev.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"fish_fillet_y.svg", dpi=300, transparent=False, bbox_inches='tight')

plt.close()

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))

counts_tot=counts_256

#masking
counts_tot_masked = np.ma.masked_array(counts_tot, counts_tot < 2)

# cs=ax.contourf(tof,ymm,fish_fillet_y_masked.T, levels=20)
cs=ax.pcolormesh(ymm_pos,xmm_pos,counts_tot_masked.T, shading='gouraud')

# ax.set_title("fish fillet y-axis")
ax.set_xlabel('x-axis [mm]')
ax.set_xlim([-62., 62])
ax.set_ylabel('y-axis [mm]')
ax.set_ylim([-62., 62.])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([fish_fillet_y_masked.min(),fish_fillet_y_masked.max()])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"position_en256ev.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"fish_fillet_y.svg", dpi=300, transparent=False, bbox_inches='tight')

plt.close()

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))

counts_tot=counts_260

#masking
counts_tot_masked = np.ma.masked_array(counts_tot, counts_tot < 2)

# cs=ax.contourf(tof,ymm,fish_fillet_y_masked.T, levels=20)
cs=ax.pcolormesh(ymm_pos,xmm_pos,counts_tot_masked.T, shading='gouraud')

# ax.set_title("fish fillet y-axis")
ax.set_xlabel('x-axis [mm]')
ax.set_xlim([-62., 62])
ax.set_ylabel('y-axis [mm]')
ax.set_ylim([-62., 62.])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([fish_fillet_y_masked.min(),fish_fillet_y_masked.max()])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"position_en260ev.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"fish_fillet_y.svg", dpi=300, transparent=False, bbox_inches='tight')

# plt.close()

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))

#masking
fish_fillet_y_masked = np.ma.masked_array(fish_fillet_y, fish_fillet_y < 25)

# cs=ax.contourf(tof,ymm,fish_fillet_y_masked.T, levels=20)
cs=ax.pcolormesh(tof,ymm,fish_fillet_y_masked.T, shading='gouraud')

# ax.set_title("fish fillet y-axis")

ax.set_xlabel('tof [ns]')
ax.set_xlim([14.75, 17.4])
ax.set_ylabel('y-axis [mm]')
ax.set_ylim([-40., 40.])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([fish_fillet_y_masked.min(),fish_fillet_y_masked.max()])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"fish_fillet_y.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"fish_fillet_y.svg", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))

#masking
fish_fillet_x_masked = np.ma.masked_array(fish_fillet_x, fish_fillet_x < 25)

# cs=ax.contourf(tof,ymm,fish_fillet_x_masked.T, levels=20)
cs=ax.pcolormesh(tof,ymm,fish_fillet_x_masked.T, shading='gouraud')

# ax.set_title("fish fillet y-axis")

ax.set_xlabel('tof [ns]')
ax.set_xlim([14.75, 17.4])
ax.set_ylabel('y-axis [mm]')
ax.set_ylim([-40., 40.])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([fish_fillet_x_masked.min(),fish_fillet_x_masked.max()])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"fish_fillet_x.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"fish_fillet_x.svg", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))

cs=ax.contourf(tof,ymm,fish_fillet_y.T, vmin=20, extend = "min")
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('tof [ns]')
ax.set_xlim([14.75, 17.4])
ax.set_ylabel('y-axis [mm]')
ax.set_ylim([-40., 40.])
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)

# cbar.set_ticklabels(["min","max"])
# cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
cbar.ax.set_ylabel('counts')
cs.cmap.set_under(color="k",alpha=0.2)


# fig.savefig(dir+"fish_fillet_y.png", figsize=(50, 24), dpi=144, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"fish_fillet_y.svg", figsize=(50, 24), dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))

cthetaEE_masked = np.ma.masked_array(cthetaEE, cthetaEE < 10)

# cs=ax.contourf(ctheta_en,en,cthetaEE_masked.T, levels=20)
cs=ax.pcolormesh(ctheta_en,en,cthetaEE_masked.T, shading='gouraud')

# ax.set_title("fish fillet y-axis")
ax.set_xlabel('cos\u03D1 [adm]')
ax.set_ylabel('kinetc energy photoelectrons [eV]')
ax.set_ylim([0., 15.])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([cthetaEE_masked.min(),cthetaEE_masked.max()])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"ctheta_EE.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"ctheta_EE.svg", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
cthetaEE_proj=np.sum(cthetaEE,axis=0)

cthetaEE_proj_masked = np.ma.masked_array(cthetaEE_proj, cthetaEE_proj < 10)

fig,ax= plt.subplots(1, figsize=(12, 10))
ax.set_xlabel('energy [eV]')
ax.set_ylabel('counts')

ax.set_box_aspect(1)

ax.plot(en,cthetaEE_proj_masked)

ax.legend(loc="best",fontsize="small")
ax.grid(linestyle = '-', linewidth = 2 , alpha = 0.4)

# fig.savefig(dir+"el_sum_wlayer_proj.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer_proj.svg", dpi=300, transparent=False, bbox_inches='tight')